## 💾데이터 전처리 실습_2: 
**❗목표:** 월별 방문자 데이터를 범용적으로 분석 가능한 Long 형태로 변환

##### [원본: 04_중앙박물관_24-25년_월별_관람객](../data/raw/museum/csv/03_중앙박물관_7개년_관람객.csv)
##### [정제: museum_visitors_2425](../data/processed/museum_visitors_2425.csv)

## Step 1: 라이브러리 import & 파일 읽기

In [1]:
import pandas as pd

# 파일 읽기
df = pd.read_csv(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\04_중앙박물관_24-25년_월별_관람객.csv')

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n원본 shape: {df.shape}")

원본 데이터:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
0,1월,325127,317286,7841,513262,505282,7980
1,2월,305525,296323,9202,543361,536035,7326
2,3월,222265,205603,16662,341471,325928,15543
3,4월,226589,205456,21133,348114,324329,23785
4,5월,286491,265843,20648,495384,472680,22704
5,6월,283530,264058,19472,474731,454084,20647
6,7월,361493,339913,21580,747679,720949,26730
7,8월,428217,408391,19826,864977,842049,22928
8,9월,351733,333975,17758,409025,384647,24378
9,10월,310040,288462,21578,553584,525901,27683



원본 shape: (13, 7)


## Step 2: 합계 행 제거

In [2]:
# 합계 행 제거 전 확인
print("제거 전 마지막 행:")
display(df.tail(2))

# 합계 행 제거
df = df[df['월'] != '합계'].copy()

print("\n제거 후 마지막 행:")
display(df.tail(2))
print(f"제거 후 shape: {df.shape}")

제거 전 마지막 행:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
11,12월,422114,412872,9242,693218,680668,12550
12,합계,3788785,3590700,198085,6507483,6276291,231192



제거 후 마지막 행:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
10,11월,265661,252518,13143,522677,503739,18938
11,12월,422114,412872,9242,693218,680668,12550


제거 후 shape: (12, 7)


## Step 3: 총계 컬럼 확인 & 제거

In [3]:
# 현재 컬럼 확인
print("현재 컬럼 목록:")
print(df.columns.tolist())

현재 컬럼 목록:
['월', '2024_총계', '2024_내국인', '2024_외국인', '2025_총계', '2025_내국인', '2025_외국인']


In [4]:
# 총계 컬럼 제거
df = df.drop(columns=['2024_총계', '2025_총계'])

print("\n총계 제거 후 컬럼:")
print(df.columns.tolist())
display(df.head(3))


총계 제거 후 컬럼:
['월', '2024_내국인', '2024_외국인', '2025_내국인', '2025_외국인']


,월,2024_내국인,2024_외국인,2025_내국인,2025_외국인
0,1월,317286,7841,505282,7980
1,2월,296323,9202,536035,7326
2,3월,205603,16662,325928,15543


## Step 4: Wide → Long 변환

In [5]:
# Wide to Long 변환
df_long = pd.melt(
    df,
    id_vars=['월'],              # 월 컬럼을 기준으로
    var_name='구분',             # 컬럼명이 '구분'으로
    value_name='방문자수'        # 값이 '방문자수'로
)

print("Long 형태 변환 결과:")
display(df_long.head(10))
print(f"\nLong 형태 shape: {df_long.shape}")

Long 형태 변환 결과:


,월,구분,방문자수
0,1월,2024_내국인,317286
1,2월,2024_내국인,296323
2,3월,2024_내국인,205603
3,4월,2024_내국인,205456
4,5월,2024_내국인,265843
5,6월,2024_내국인,264058
6,7월,2024_내국인,339913
7,8월,2024_내국인,408391
8,9월,2024_내국인,333975
9,10월,2024_내국인,288462



Long 형태 shape: (48, 3)


In [6]:
# split 후 rename으로 컬럼명 변경
temp = df_long['구분'].str.split('_', expand=True)
temp = temp.rename(columns={0: '연도', 1: '카테고리'})

# 원본에 합치기
df = pd.concat([df_long, temp], axis=1)

# '구분' 컬럼 삭제
df = df.drop('구분', axis=1)

# 컬럼 순서 변경 (연도, 카테고리를 앞으로)
df = df[['연도', '월','카테고리','방문자수']]

display(df.style.format({'방문자수': '{:,}'}))  # 원본 유지

,연도,월,카테고리,방문자수
0,2024,1월,내국인,"317,286"
1,2024,2월,내국인,"296,323"
2,2024,3월,내국인,"205,603"
3,2024,4월,내국인,"205,456"
4,2024,5월,내국인,"265,843"
5,2024,6월,내국인,"264,058"
6,2024,7월,내국인,"339,913"
7,2024,8월,내국인,"408,391"
8,2024,9월,내국인,"333,975"
9,2024,10월,내국인,"288,462"


## Step 6: 데이터 타입 변환

In [7]:
# 현재 데이터 타입 확인
print("변환 전 데이터 타입:")
print(df.dtypes)

변환 전 데이터 타입:
연도      object
월       object
카테고리    object
방문자수     int64
dtype: object


In [8]:
# 숫자형으로 변환
df['방문자수'] = pd.to_numeric(df['방문자수'])
df['연도'] = df['연도'].astype(int)
df['월'] = df['월'].str.replace('월', '').astype(int)

In [9]:
print("\n변환 후 데이터 타입:")
print(df.dtypes)
print("\n변환 결과 확인:")
display(df.head(6))


변환 후 데이터 타입:
연도       int64
월        int64
카테고리    object
방문자수     int64
dtype: object

변환 결과 확인:


,연도,월,카테고리,방문자수
0,2024,1,내국인,317286
1,2024,2,내국인,296323
2,2024,3,내국인,205603
3,2024,4,내국인,205456
4,2024,5,내국인,265843
5,2024,6,내국인,264058


## Step 7: 날짜 컬럼 생성

In [10]:
# 날짜 컬럼 생성
df['날짜']=pd.to_datetime(
    df['연도'].astype(str)+'-'+
    df['월'].astype(str).str.zfill(2) + '-01'
)
print("날짜 컬럼 생성 결과:")
display(df.head(20))
print(f"\n날짜 데이터 타입: {df['날짜'].dtype}")

날짜 컬럼 생성 결과:


,연도,월,카테고리,방문자수,날짜
0,2024,1,내국인,317286,2024-01-01
1,2024,2,내국인,296323,2024-02-01
2,2024,3,내국인,205603,2024-03-01
3,2024,4,내국인,205456,2024-04-01
4,2024,5,내국인,265843,2024-05-01
5,2024,6,내국인,264058,2024-06-01
6,2024,7,내국인,339913,2024-07-01
7,2024,8,내국인,408391,2024-08-01
8,2024,9,내국인,333975,2024-09-01
9,2024,10,내국인,288462,2024-10-01



날짜 데이터 타입: datetime64[ns]


## Step 8: 컬럼 순서 정리 & 정렬

In [11]:
df.dtypes

연도               int64
월                int64
카테고리            object
방문자수             int64
날짜      datetime64[ns]
dtype: object

In [12]:
# 컬럼명 변경 및 순서 정리

df = df.rename(columns={
    '연도': 'year',
    '월': 'month',
    '카테고리': 'category',
    '날짜': 'date',
    '방문자수': 'visitors'
})

display(df.head())

,year,month,category,visitors,date
0,2024,1,내국인,317286,2024-01-01
1,2024,2,내국인,296323,2024-02-01
2,2024,3,내국인,205603,2024-03-01
3,2024,4,내국인,205456,2024-04-01
4,2024,5,내국인,265843,2024-05-01


In [13]:

# 정렬
df = df.sort_values(['date', 'category'], ascending=[True, True]).reset_index(drop=True)

print("최종 결과 :")
display(df.style.format({'visitors': '{:,}'}))
print(f"\n최종 shape: {df.shape}")
print(f"카테고리 종류: {df['category'].unique()}")

최종 결과 :


,year,month,category,visitors,date
0,2024,1,내국인,"317,286",2024-01-01 00:00:00
1,2024,1,외국인,"7,841",2024-01-01 00:00:00
2,2024,2,내국인,"296,323",2024-02-01 00:00:00
3,2024,2,외국인,"9,202",2024-02-01 00:00:00
4,2024,3,내국인,"205,603",2024-03-01 00:00:00
5,2024,3,외국인,"16,662",2024-03-01 00:00:00
6,2024,4,내국인,"205,456",2024-04-01 00:00:00
7,2024,4,외국인,"21,133",2024-04-01 00:00:00
8,2024,5,내국인,"265,843",2024-05-01 00:00:00
9,2024,5,외국인,"20,648",2024-05-01 00:00:00



최종 shape: (48, 5)
카테고리 종류: ['내국인' '외국인']


## Step 9: 기본 확인 (결측치, 통계)

In [14]:
# 결측치 확인
print("결측치 개수:")
print(df.isnull().sum())

# 기본 통계
print("\n방문자수 기본 통계:")
display(df['visitors'].describe())

# 카테고리별 합계
print("\n카테고리별 총 방문자수:")
display(df.groupby('category')['visitors'].sum())
df.index.name = 'id'
display(df)

결측치 개수:
year        0
month       0
category    0
visitors    0
date        0
dtype: int64

방문자수 기본 통계:


count        48.000000
mean     214505.583333
std      229648.001766
min        7326.000000
25%       19737.500000
50%      116569.500000
75%      351096.500000
max      842049.000000
Name: visitors, dtype: float64


카테고리별 총 방문자수:


category
내국인    9866991
외국인     429277
Name: visitors, dtype: int64

,year,month,category,visitors,date
id,,,,,
0,2024,1,내국인,317286,2024-01-01
1,2024,1,외국인,7841,2024-01-01
2,2024,2,내국인,296323,2024-02-01
3,2024,2,외국인,9202,2024-02-01
4,2024,3,내국인,205603,2024-03-01
5,2024,3,외국인,16662,2024-03-01
6,2024,4,내국인,205456,2024-04-01
7,2024,4,외국인,21133,2024-04-01
8,2024,5,내국인,265843,2024-05-01


In [15]:
df.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_visitors_2425.csv', 
          encoding='utf-8-sig', 
          index=False)

### 넷플릭스 전처리 코드

In [20]:
import pandas as pd

In [21]:
df.columns.tolist()

['week',
 'start_date',
 'end_date',
 'rank_global',
 'rank_kr',
 'rank_us',
 'rank_en',
 'hours',
 'views',
 'cum_hours',
 'cum_views']

In [17]:
df=pd.read_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\netflix_ranking_kpdh.csv')
df

,week,start_date,end_date,rank_global,rank_kr,rank_us,rank_en,hours,views,cum_hours,cum_views
0,1,2025-06-16,2025-06-22,2,2,6,2,15400000,9200000,15400000,9200000
1,2,2025-06-23,2025-06-29,1,1,2,1,40300000,24200000,55700000,33400000
2,3,2025-06-30,2025-07-06,2,1,2,2,37800000,22700000,93500000,56100000
3,4,2025-07-07,2025-07-13,1,1,2,1,40400000,24200000,133900000,80300000
4,5,2025-07-14,2025-07-20,1,1,2,1,43000000,25800000,176900000,106100000
5,6,2025-07-21,2025-07-27,2,1,2,2,43900000,26300000,220800000,132400000
6,7,2025-07-28,2025-08-03,2,2,2,2,43800000,26300000,264600000,158700000
7,8,2025-08-04,2025-08-10,2,1,1,2,43100000,25900000,307700000,184600000
8,9,2025-08-11,2025-08-17,1,1,1,1,43300000,26000000,351000000,210600000
9,10,2025-08-18,2025-08-24,1,1,1,1,42400000,25400000,393400000,236000000


In [19]:
df = df.rename(columns={
    '주차': 'week',
    '시작일': 'start_date',
    '종료일': 'end_date',
    '전체순위': 'rank_global',
    '한국순위': 'rank_kr',
    '미국순위': 'rank_us',
    '영어순위': 'rank_en',
    '시청시간_시간': 'hours',
    '시청횟수_회': 'views',
    '누적시청시간': 'cum_hours',
    '누적시청횟수': 'cum_views'
})

# df.dtypes

# df['start_date'] = pd.to_datetime(df['start_date'])
# df['end_date'] = pd.to_datetime(df['end_date'])

# df['hours'] = df['hours'].str.replace(',', '').astype(int)
# df['views'] = df['views'].str.replace(',', '').astype(int)
# df['cum_hours'] = df['cum_hours'].str.replace(',', '').astype(int)
# df['cum_views'] = df['cum_views'].str.replace(',', '').astype(int)

df.dtypes

week                    int64
start_date     datetime64[ns]
end_date       datetime64[ns]
rank_global             int64
rank_kr                 int64
rank_us                 int64
rank_en                 int64
hours                   int64
views                   int64
cum_hours               int64
cum_views               int64
dtype: object

In [ ]:
display(df.head())

,week,start_date,end_date,rank_global,rank_kr,rank_us,rank_en,hours,views,cum_hours,cum_views
0,1,2025-06-16,2025-06-22,2,2,6,2,15400000,9200000,15400000,9200000
1,2,2025-06-23,2025-06-29,1,1,2,1,40300000,24200000,55700000,33400000
2,3,2025-06-30,2025-07-06,2,1,2,2,37800000,22700000,93500000,56100000
3,4,2025-07-07,2025-07-13,1,1,2,1,40400000,24200000,133900000,80300000
4,5,2025-07-14,2025-07-20,1,1,2,1,43000000,25800000,176900000,106100000


In [24]:
df = df[['week', 'start_date', 'end_date', 'rank_global','rank_en', 'rank_kr', 'rank_us','hours', 'views', 'cum_hours', 'cum_views']]
display(df)

,week,start_date,end_date,rank_global,rank_en,rank_kr,rank_us,hours,views,cum_hours,cum_views
0,1,2025-06-16,2025-06-22,2,2,2,6,15400000,9200000,15400000,9200000
1,2,2025-06-23,2025-06-29,1,1,1,2,40300000,24200000,55700000,33400000
2,3,2025-06-30,2025-07-06,2,2,1,2,37800000,22700000,93500000,56100000
3,4,2025-07-07,2025-07-13,1,1,1,2,40400000,24200000,133900000,80300000
4,5,2025-07-14,2025-07-20,1,1,1,2,43000000,25800000,176900000,106100000
5,6,2025-07-21,2025-07-27,2,2,1,2,43900000,26300000,220800000,132400000
6,7,2025-07-28,2025-08-03,2,2,2,2,43800000,26300000,264600000,158700000
7,8,2025-08-04,2025-08-10,2,2,1,1,43100000,25900000,307700000,184600000
8,9,2025-08-11,2025-08-17,1,1,1,1,43300000,26000000,351000000,210600000
9,10,2025-08-18,2025-08-24,1,1,1,1,42400000,25400000,393400000,236000000


In [25]:
df.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\netflix_ranking_kpdh.csv', index=False)

In [26]:
display(df.head())

,week,start_date,end_date,rank_global,rank_en,rank_kr,rank_us,hours,views,cum_hours,cum_views
0,1,2025-06-16,2025-06-22,2,2,2,6,15400000,9200000,15400000,9200000
1,2,2025-06-23,2025-06-29,1,1,1,2,40300000,24200000,55700000,33400000
2,3,2025-06-30,2025-07-06,2,2,1,2,37800000,22700000,93500000,56100000
3,4,2025-07-07,2025-07-13,1,1,1,2,40400000,24200000,133900000,80300000
4,5,2025-07-14,2025-07-20,1,1,1,2,43000000,25800000,176900000,106100000
